In [1]:
using BandedMatrices, LinearAlgebra, SparseArrays, IntervalArithmetic, Plots, Base.Threads, Serialization

In [2]:
Base.:*(A::SparseMatrixCSC{Interval{Float64}, Int64},x::Vector{Interval{Float64}})=Vector((A*sparse(x[:,:]))[:])

In [3]:
function D(N)
    # implements ∂ₓ in Fourier
    v = interval.([ ((n+1 ) % 2) * (n÷2) for n=1:2*N])
    return dropzeros(sparse(BandedMatrix(-1 => v, 1 =>-v)))
end

function D2(N)
    # implements ∂ₓₓ in Fourier
    v = interval.([-(n÷2)^2 for n=1:2*N+1])
    return dropzeros(sparse(BandedMatrix( 0 => vcat(v, zeros(Interval{Float64}, 4)))[:,1:2*N+1]))
end

function D2θ(N)
    # implements ∂ₓₓ in Fourier
    v = interval.([-(n÷2)^2 for n=1:2*N+1])
    return dropzeros(sparse(BandedMatrix( 0 => vcat(v, zeros(Interval{Float64}, 4)))[1:2*N+1,1:2*N+1]))
end

function id(N)
    v = ones(Interval{Float64}, 2*N+1)
    return dropzeros(sparse(BandedMatrix( 0 => v)))
end

function id2(N)
    v = ones(Interval{Float64}, 2*N+1)
    return dropzeros(sparse(BandedMatrix( 0 => vcat(v, zeros(Interval{Float64}, 4)))[:,1:2*N+1]))
end

function idθ(N)
    v = ones(Interval{Float64}, 2*N+1)
    return dropzeros(sparse(BandedMatrix( 0 => vcat(v, zeros(Interval{Float64}, 2)))[:,1:2*N+1]))
end

function C(N)
    # implements the u ↦ cosx u in Fourier
    v = vcat([interval(0.0)], ones(Interval{Float64}, 2*N+2)/interval(2))
    A = dropzeros(sparse(BandedMatrix( -2 => v, 2 => v[1:end-2])[:,1:2*N+1]))
    A[1,3] = interval(0.5)
    A[3,1] = interval(1.0)
    return A
end

function S(N)
    # implements the u ↦ sinx u in Fourier
    v = vcat([interval(0.0)],interval.([ ((n+1 ) % 2) for n=1:2*N+2]))/interval(2)
    A = dropzeros(sparse(BandedMatrix( -1 => v, -3 =>-v[2:end], 1 => v, 3=>-v[2:end])[:,1:2*N+1]))
    A[1,2] = interval(0.5)
    A[2,1] = interval(1.0)
    return A
end

function C2(N)
    # implements the u ↦ cos2x u in Fourier
    v = vcat([interval(0.0)], ones(Interval{Float64}, 2*N)/interval(2))
    A = dropzeros(sparse(BandedMatrix( -4 => v, 4 => v[2:end-1])[:,1:2*N+1]))
    # A[1,3] = interval(0.5)
    A[5,1] = interval(1.0)
    A[3,3] = interval(0.5)
    A[2,2] = -interval(0.5)
    return A
end

function S2(N)
    # implements the u ↦ sin2x u in Fourier
    v = vcat([interval(0.0)],interval.([ ((n+1 ) % 2) for n=1:2*N]))/interval(2)
    A = dropzeros(sparse(BandedMatrix( -3 => v, -5 =>-v[2:end], 3 => v, 5=>-v[2:end])[:,1:2*N+1]))
    A[1,4] = interval(0.5)
    A[4,1] = interval(1.0)
    A[2,3] = interval(0.5)
    A[3,2] = interval(0.5)
    return A
end

S2 (generic function with 1 method)

In [4]:
N = 25
M = 500;

In [5]:
Dx = ((kron(S2(N),id2(N),id2(M))/interval(4)-kron(S(N),C(N),id2(M)))*kron(D(N),id(N),id(M)));

In [6]:
Dy = ((kron(id2(N),S2(N),id2(M))/interval(4)+kron(C(N),S(N),id2(M)))*kron(id(N),D(N),id(M)));

In [7]:
Dθ = (((interval(2)*(-kron(S(N),S(N),id2(M))+kron(kron(id2(N),C2(N))/interval(4)+kron(C(N),C(N))-kron(C2(N),id2(N))/interval(4), S(M)))+kron(id2(N), id2(N), S2(M)))*kron(id(N),id(N),D(M))));

In [8]:
Δ = (kron(D2(N),id2(N),id2(M))+kron(id2(N),D2(N),id2(M)));

In [9]:
Δθ = (kron(id2(N), id2(N), id2(M)-C2(M))*kron(id(N), id(N), D2θ(M)));

In [10]:
Q = Vector((kron(kron(C2(N),id2(N))+kron(id2(N),C2(N)),id2(M))/interval(4)+kron(kron(C2(N), id2(N))/interval(4)-kron(C(N),C(N))-kron(id2(N),C2(N))/interval(4),C(M))-kron(id2(N),id2(N),id2(M)+C2(M))/interval(2))[:,1]);

In [11]:
𝔏 = (Dx + Dy + Dθ + Δ +Δθ)

3040125×2603601 SparseMatrixCSC{Interval{Float64}, Int64} with 77881299 stored entries:
⎡⣿⣿⢳⣽⣦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⣙⣾⣿⣟⣾⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠈⠻⣽⣿⡿⣽⣿⡷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠘⢧⣿⣟⣷⣿⢿⣳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠙⢾⡿⣿⣾⡿⣏⣧⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠙⢿⣻⣽⣿⣿⣽⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠹⣟⣷⣿⢿⣳⣿⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣽⣾⡿⣿⣿⡿⣆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣿⣿⣻⣽⣿⣳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⢳⡿⢿⣷⣿⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢾⡿⣽⣿⡟⣯⣷⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣳⣿⣿⣳⣿⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢯⣷⣿⢯⣷⣿⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠻⣬⣿⡟⣯⣿⣟⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣿⢿⣻⣿⢿⣹⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣿⢯⣿⣿⢯⣷⣄⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⢿⣛⣾⡿⣟⣾⣷⣄⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣿⢿⣹⣾⠷⡄⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢯⣿⣿⣯⣿⣿⣦⡀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢾⡿⣟⣾⡿⣟⡦⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣿⢿⣿⣿⠻⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀

In [12]:
ū = Vector(interval.(deserialize("baxendale_ubar")));
ϵ = 𝔏*ū-Q
λ̄ = -ϵ[1]
ϵ[1] = interval(0)
δ = sum(abs.(ϵ))
λ = λ̄ +interval(-1,1)*δ

[-0.613919, -0.610938]_com